In [12]:
import pandas as pd
from sqlalchemy import create_engine
from functools import reduce
from credentials import user
from credentials import server
from credentials import password

from credentials import user, password, server

In [13]:
engine = create_engine("postgres+psycopg2://"+user+":"+password+"@"+server+"5432/covid_data")

In [14]:
service_requests_df = pd.read_csv('C:/Users/Natalia/Documents/NSS/class_projects/nashville-covid-quaranteam/data/hubNashville_311_Service_Requests.csv', low_memory=False)
clusters_df = pd.read_csv('C:/Users/Natalia/Documents/NSS/class_projects/nashville-covid-quaranteam/data/clusters.csv', low_memory=False)
businesses_df = pd.read_csv('C:/Users/Natalia/Documents/NSS/class_projects/nashville-covid-quaranteam/data/businesses.csv', low_memory=False)

In [15]:
service_requests_df.head(2)

,Request #,Status,Request Type,Subrequest Type,Additional Subrequest Type,Date / Time Opened,Date / Time Closed,Request Origin,System of Origin ID,Contact Type,...,State Issue,Parent Request,Closed When Created,Address,City,Council District,ZIP,Latitude,Longitude,Mapped Location
0,113016,Closed,"Trash, Recycling & Litter",Cart Service,Trash - Request an Additional Cart,10/30/2018 01:47:00 PM,03/05/2019 09:35:00 PM,Phone,NaN,NaN,...,False,NaN,False,2009 Upland Dr,NASHVILLE,7.0,37216,36.198638,-86.706659,"(36.1986378, -86.7066586)"
1,53502,Closed,Property Violations,Residential Complaint,Residential Complaint,03/13/2018 09:34:50 PM,04/06/2018 01:13:32 PM,Phone,NaN,NaN,...,False,NaN,False,631 Murfreesboro Pike,NASHVILLE,17.0,37210,36.140070,-86.738360,"(36.14007, -86.73836)"


In [16]:
clusters_df.head(2)

,Cluster Name,Facility Type,Cluster Start Date,# Cases,Latitude,Longitude
0,Vanderbilt Parties,Social Gathering,3/11/2020,49,36.144703,-86.802655
1,Event at Clementine Hall,Social Gathering,3/14/2020,23,36.152444,-86.844583


In [17]:
businesses_df.head(2)

,address,vicinity,name,type
0,"2440 COUCHVILLE PIKE, Nashville","2440 Couchville Pike, Nashville",Wilson Sporting Goods Co,"['point_of_interest', 'establishment']"
1,"1613 Buchanan St, Nashville","1613 Buchanan Street, Nashville",Coin Cloud Bitcoin ATM,"['atm', 'finance', 'point_of_interest', 'estab..."


In [18]:
service_requests_filtered = service_requests_df[['Request Type','Subrequest Type','Date / Time Opened','Address', 'City','ZIP','Latitude','Longitude','Mapped Location']]
service_requests_filtered.head(2)

,Request Type,Subrequest Type,Date / Time Opened,Address,City,ZIP,Latitude,Longitude,Mapped Location
0,"Trash, Recycling & Litter",Cart Service,10/30/2018 01:47:00 PM,2009 Upland Dr,NASHVILLE,37216,36.198638,-86.706659,"(36.1986378, -86.7066586)"
1,Property Violations,Residential Complaint,03/13/2018 09:34:50 PM,631 Murfreesboro Pike,NASHVILLE,37210,36.140070,-86.738360,"(36.14007, -86.73836)"


In [20]:
service_requests_dropped_nan = service_requests_filtered.dropna()
service_requests_dropped_nan.head(2)

,Request Type,Subrequest Type,Date / Time Opened,Address,City,ZIP,Latitude,Longitude,Mapped Location
0,"Trash, Recycling & Litter",Cart Service,10/30/2018 01:47:00 PM,2009 Upland Dr,NASHVILLE,37216,36.198638,-86.706659,"(36.1986378, -86.7066586)"
1,Property Violations,Residential Complaint,03/13/2018 09:34:50 PM,631 Murfreesboro Pike,NASHVILLE,37210,36.140070,-86.738360,"(36.14007, -86.73836)"


In [22]:
service_requests_renamed = service_requests_dropped_nan.rename(columns={'Request Type':'req_type', 'Subrequest Type':'subreq_type','Date / Time Opened':'date_time','Address':'address','City':'city','ZIP':'zip','Latitude':'lat','Longitude':'long','Mapped Location':'mapped_loc'})
service_requests_renamed.head(2)

,req_type,subreq_type,date_time,address,city,zip,lat,long,mapped_loc
0,"Trash, Recycling & Litter",Cart Service,10/30/2018 01:47:00 PM,2009 Upland Dr,NASHVILLE,37216,36.198638,-86.706659,"(36.1986378, -86.7066586)"
1,Property Violations,Residential Complaint,03/13/2018 09:34:50 PM,631 Murfreesboro Pike,NASHVILLE,37210,36.140070,-86.738360,"(36.14007, -86.73836)"


In [23]:
#covid_violations_renamed[['date','time']] = covid_violations_renamed.date_time.str.split(' ',expand=True)
service_requests_renamed['date'] = service_requests_renamed.date_time.str.split(' ', expand = True)[0]
service_requests_renamed['time'] = service_requests_renamed.date_time.str.split(' ', expand = True)[1]
service_requests_renamed ['AM/PM'] = service_requests_renamed.date_time.str.split(' ', expand = True)[2]
service_requests_renamed.head(2)

,req_type,subreq_type,date_time,address,city,zip,lat,long,mapped_loc,date,time,AM/PM
0,"Trash, Recycling & Litter",Cart Service,10/30/2018 01:47:00 PM,2009 Upland Dr,NASHVILLE,37216,36.198638,-86.706659,"(36.1986378, -86.7066586)",10/30/2018,01:47:00,PM
1,Property Violations,Residential Complaint,03/13/2018 09:34:50 PM,631 Murfreesboro Pike,NASHVILLE,37210,36.140070,-86.738360,"(36.14007, -86.73836)",03/13/2018,09:34:50,PM


In [24]:
##covid_violations_renamed['date'] = covid_violations_renamed.date_time.str.split().str[0]
##covid_violations_renamed.head(2)

In [27]:
covid_violations = service_requests_renamed[service_requests_renamed["req_type"]== 'COVID-19']
covid_violations = covid_violations.reset_index(drop=True)
covid_violations.head()

,req_type,subreq_type,date_time,address,city,zip,lat,long,mapped_loc,date,time,AM/PM
0,COVID-19,COVID-19 Violations,08/14/2020 02:38:00 AM,2045 Antioch Pike,ANTIOCH,37013,36.068546,-86.683898,"(36.0685457, -86.6838975)",08/14/2020,02:38:00,AM
1,COVID-19,COVID-19 Questions,06/18/2020 08:59:50 PM,Charlotte Pike,NASHVILLE,37209,36.114870,-86.925890,"(36.11487, -86.92589)",06/18/2020,08:59:50,PM
2,COVID-19,COVID-19 Violations,10/13/2020 11:48:40 AM,2800 Opryland Dr,NASHVILLE,37214,36.211269,-86.694302,"(36.211269429469425, -86.6943024067459)",10/13/2020,11:48:40,AM
3,COVID-19,COVID-19 Violations,06/16/2020 12:35:20 AM,2607 12th Ave S,NASHVILLE,37204,36.123500,-86.790130,"(36.1235, -86.79013)",06/16/2020,12:35:20,AM
4,COVID-19,COVID-19 Questions,06/18/2020 03:13:10 PM,2431 Gallatin Pike N,MADISON,37115,36.308564,-86.675032,"(36.3085641, -86.67503239999999)",06/18/2020,03:13:10,PM


##covid_violations.date_time.str.split(' ', 1)[0][1]

date_list = []
time_list = []

for i in range(len(covid_violations)):
    time_list.append(covid_violations.date_time.str.split(' ', 1)[i][1])
    print(date_list)

[]
